In [11]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Jul  4 08:29:24 2020
@author: koyama

"""
import os
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn import cluster

#Setting directly
os.chdir("C:/Users/prajj/OneDrive/Desktop/capstoneserver/model/Predicting-sensory-evaluation-of-spinach-freshness-using-machine-learning-model-and-digital-images/Attached_file")
#Loading label
df_label = pd.read_csv('data/label_all.csv', names=['label'])
#Vector
label = df_label.values
label_all = label.flatten()
#All data set
N_img =1045
#Colum_Feature
col_color=[]
col_color.extend(['gry', 'b', 'g', 'r', 'h', 's', 'v', 'l', 'a', 'bl',
                  'gry_std','b_std', 'g_std', 'r_std','h_std', 's_std', 'v_std','l_std','a_std','bl_std',
                  'gry_min','b_min','g_min','r_min','h_min','s_min','v_min', 'l_min','a_min','bl_min'])
#Matrix_feature
feature_matrix_color = np.zeros((N_img,len(col_color)))
feature_color = pd.DataFrame(feature_matrix_color, columns = col_color)
#Preparing_for_features
#Color information and shape
for i in range(N_img):
    filename = "image/"+str(i+1).zfill(4)+".jpg"
    img = cv2.imread(filename)
    img = cv2.resize(img, (int(np.round(img.shape[1]/5)), int(np.round(img.shape[0]/5))))
    gry =cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #Otsu method
    ret, img_thresh = cv2.threshold(gry, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((3,3),np.uint8)
    img_thresh = cv2.morphologyEx(img_thresh, cv2.MORPH_OPEN, kernel)
    img_thresh = cv2.bitwise_not(img_thresh)
    img_thresh = cv2.erode(img_thresh,kernel,iterations = 1)
    height, width, color = img.shape 
    img_thresh3 = np.zeros((height, width, 3), dtype = "uint8")
    img_thresh3[:,:,0], img_thresh3[:,:,1], img_thresh3[:,:,2] = img_thresh/255, img_thresh/255, img_thresh/255
    #Remove Background 
    img_Hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    img_Lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    dst_gry = gry * (img_thresh/255)
    dst_bgr = img * img_thresh3
    dst_Hsv = img_Hsv * img_thresh3
    dst_Lab = img_Lab * img_thresh3
    
    #Color feature extraction: gry,bgr,hsv,lab mean,std,min
    feature_color['gry'][i] = dst_gry.T.flatten()[np.nonzero(dst_gry.T.flatten())].mean()
    feature_color['b'][i] = dst_bgr.T[0].flatten()[np.nonzero(dst_bgr.T[0].flatten())].mean()
    feature_color['g'][i] = dst_bgr.T[1].flatten()[np.nonzero(dst_bgr.T[1].flatten())].mean()
    feature_color['r'][i] = dst_bgr.T[2].flatten()[np.nonzero(dst_bgr.T[2].flatten())].mean()
    feature_color['h'][i] = dst_Hsv.T[0].flatten()[np.nonzero(dst_Hsv.T[0].flatten())].mean()
    feature_color['s'][i] = dst_Hsv.T[1].flatten()[np.nonzero(dst_Hsv.T[1].flatten())].mean()
    feature_color['v'][i] = dst_Hsv.T[2].flatten()[np.nonzero(dst_Hsv.T[2].flatten())].mean()
    feature_color['l'][i] = dst_Lab.T[0].flatten()[np.nonzero(dst_Lab.T[0].flatten())].mean()
    feature_color['a'][i] = dst_Lab.T[1].flatten()[np.nonzero(dst_Lab.T[1].flatten())].mean()
    feature_color['bl'][i] = dst_Lab.T[2].flatten()[np.nonzero(dst_Lab.T[2].flatten())].mean()
    feature_color['gry_std'][i] = dst_gry.T.flatten()[np.nonzero(dst_gry.T.flatten())].std()
    feature_color['b_std'][i] = dst_bgr.T[0].flatten()[np.nonzero(dst_bgr.T[0].flatten())].std()
    feature_color['g_std'][i] = dst_bgr.T[1].flatten()[np.nonzero(dst_bgr.T[1].flatten())].std()
    feature_color['r_std'][i] = dst_bgr.T[2].flatten()[np.nonzero(dst_bgr.T[2].flatten())].std()
    feature_color['h_std'][i] = dst_Hsv.T[0].flatten()[np.nonzero(dst_Hsv.T[0].flatten())].std()
    feature_color['s_std'][i] =  dst_Hsv.T[1].flatten()[np.nonzero(dst_Hsv.T[1].flatten())].std()
    feature_color['v_std'][i] = dst_Hsv.T[2].flatten()[np.nonzero(dst_Hsv.T[2].flatten())].std()
    feature_color['l_std'][i] = dst_Lab.T[0].flatten()[np.nonzero(dst_Lab.T[0].flatten())].std()
    feature_color['a_std'][i] = dst_Lab.T[1].flatten()[np.nonzero(dst_Lab.T[1].flatten())].std()
    feature_color['bl_std'][i] = dst_Lab.T[2].flatten()[np.nonzero(dst_Lab.T[2].flatten())].std()
    feature_color['gry_min'][i] = dst_gry.T.flatten()[np.nonzero(dst_gry.T.flatten())].min()
    feature_color['b_min'][i] = dst_bgr.T[0].flatten()[np.nonzero(dst_bgr.T[0].flatten())].min()
    feature_color['g_min'][i] = dst_bgr.T[1].flatten()[np.nonzero(dst_bgr.T[1].flatten())].min()
    feature_color['r_min'][i] = dst_bgr.T[2].flatten()[np.nonzero(dst_bgr.T[2].flatten())].min()
    feature_color['h_min'][i] = dst_Hsv.T[0].flatten()[np.nonzero(dst_Hsv.T[0].flatten())].min()
    feature_color['s_min'][i] = dst_Hsv.T[1].flatten()[np.nonzero(dst_Hsv.T[1].flatten())].min()
    feature_color['v_min'][i] = dst_Hsv.T[2].flatten()[np.nonzero(dst_Hsv.T[2].flatten())].min()
    feature_color['l_min'][i] = dst_Lab.T[0].flatten()[np.nonzero(dst_Lab.T[0].flatten())].min()
    feature_color['a_min'][i] = dst_Lab.T[1].flatten()[np.nonzero(dst_Lab.T[1].flatten())].min()
    feature_color['bl_min'][i] = dst_Lab.T[2].flatten()[np.nonzero(dst_Lab.T[2].flatten())].min()

#label_color_fearue
df_att = pd.concat([df_label[:N_img],feature_color],axis=1)
#Save the label and color information
df_att.to_csv('data/color_features_dummy.csv')
#Load the label and color information
df = pd.read_csv('data/color_features_dummy.csv', index_col=0)
#Seperating the dataset as response variable and feature variabes
X = df.drop(['label'],axis=1)
y = df['label']
#Preparing for local feature by Orb
#Train and Test splitting of data 
X_train=X
y_train=y
index_train = X_train.index
#Preparing_for_local_feature_detection
orb = cv2.ORB_create()
features_orb = []
features_orb_bgr = [[],[],[]]
features_orb_Lab = [[], [], []]
features_orb_hsv = [[], [], []]
for i in index_train:
    filename = "image/"+str(i+1).zfill(4)+".jpg"
    img = cv2.imread(filename)
    img = cv2.resize(img, (int(np.round(img.shape[1]/5)), int(np.round(img.shape[0]/5))))
    gry =cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, img_thresh = cv2.threshold(gry, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((3,3),np.uint8)
    img_thresh = cv2.morphologyEx(img_thresh, cv2.MORPH_OPEN, kernel)
    img_thresh = cv2.bitwise_not(img_thresh)
    img_thresh = cv2.erode(img_thresh,kernel,iterations = 1)
    height, width, color = img.shape         
    img_thresh3 = np.zeros((height, width, 3), dtype = "uint8")
    img_thresh3[:,:,0], img_thresh3[:,:,1], img_thresh3[:,:,2] = img_thresh/255, img_thresh/255, img_thresh/255
    img_Lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    img_Hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    dst_bgr = img * img_thresh3
    dst_Lab = img_Lab * img_thresh3
    dst_Hsv = img_Hsv * img_thresh3
    try:
        features_orb.extend(orb.detectAndCompute(gry, None)[1])
    except:
        pass
    
    for z in range(3):
        try:
            features_orb_bgr[z].extend(orb.detectAndCompute(dst_bgr[:,:,z], None)[1])
        except:
            pass
        try:
            features_orb_Lab[z].extend(orb.detectAndCompute(dst_Lab[:,:,z], None)[1])
        except:
            pass    
        try:
            features_orb_hsv[z].extend(orb.detectAndCompute(dst_Hsv[:,:,z], None)[1])
        except:
            pass    
#Making bag of features
c_n = 20
visual_words_orb_bgr = [[],[],[]]
visual_words_orb_Lab = [[],[],[]]
visual_words_orb_hsv = [[],[],[]]
visual_words_orb = cluster.MiniBatchKMeans(n_clusters=c_n,random_state=25).fit(features_orb).cluster_centers_
for z in range(3):
    visual_words_orb_bgr[z] = cluster.MiniBatchKMeans(n_clusters=c_n,random_state=25).fit(features_orb_bgr[z]).cluster_centers_
    visual_words_orb_Lab[z] = cluster.MiniBatchKMeans(n_clusters=c_n,random_state=25).fit(features_orb_Lab[z]).cluster_centers_
    visual_words_orb_hsv[z] = cluster.MiniBatchKMeans(n_clusters=c_n,random_state=25).fit(features_orb_hsv[z]).cluster_centers_

#Training
#Vector of visual word index for training data set
Vector_orb_gry = np.zeros([len(index_train),c_n])
Vector_orb_bgr = [np.zeros([len(index_train),c_n]),np.zeros([len(index_train),c_n]),np.zeros([len(index_train),c_n])]
Vector_orb_Lab = [np.zeros([len(index_train),c_n]),np.zeros([len(index_train),c_n]),np.zeros([len(index_train),c_n])]
Vector_orb_hsv = [np.zeros([len(index_train),c_n]),np.zeros([len(index_train),c_n]),np.zeros([len(index_train),c_n])]
for i, I  in zip(range(len(index_train)),index_train):
    filename = "image/"+str(I+1).zfill(4)+".jpg"
    img = cv2.imread(filename)
    img = cv2.resize(img, (int(np.round(img.shape[1]/5)), int(np.round(img.shape[0]/5))))
    gry =cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, img_thresh = cv2.threshold(gry, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((3,3),np.uint8)
    img_thresh = cv2.morphologyEx(img_thresh, cv2.MORPH_OPEN, kernel)
    img_thresh = cv2.bitwise_not(img_thresh)
    kernel = np.ones((3,3),np.uint8)
    img_thresh = cv2.erode(img_thresh,kernel,iterations = 1)
    height, width, color = img.shape 
    dst_bgr = np.zeros((height, width, 3), dtype = "uint8")             
    img_thresh3 = np.zeros((height, width, 3), dtype = "uint8")
    img_thresh3[:,:,0], img_thresh3[:,:,1], img_thresh3[:,:,2] = img_thresh/255, img_thresh/255, img_thresh/255
    img_Lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    img_Hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    dst_bgr = img * img_thresh3
    dst_Hsv = img_Hsv * img_thresh3
    dst_Lab = img_Lab * img_thresh3
    
    #orb gry
    features_orb = orb.detectAndCompute(gry, None)[1]
    vector_orb = np.zeros(c_n)
    try:
        for f in features_orb:
            vector_orb[((visual_words_orb - f)**2).sum(axis=1).argmin()] += 1
    except:
            pass
    Vector_orb_gry[i,:] = vector_orb
    #Orb color
    for z in range(3):
        #orb bgr
        features_orb = orb.detectAndCompute(dst_bgr[:,:,z], None)[1]
        vector_orb = np.zeros(c_n)
        try:
            for f in features_orb:
                vector_orb[((visual_words_orb_bgr[z] - f)**2).sum(axis=1).argmin()] += 1
        except:
                pass
        Vector_orb_bgr[z][i,:] = vector_orb
        
        #orb Lab
        features_orb = orb.detectAndCompute(dst_Lab[:,:,z], None)[1]
        vector_orb = np.zeros(c_n)
        try:
            for f in features_orb:
                vector_orb[((visual_words_orb_Lab[z] - f)**2).sum(axis=1).argmin()] += 1
        except:
                pass
        Vector_orb_Lab[z][i,:] = vector_orb
        
        #orb hsv
        features_orb = orb.detectAndCompute(dst_Hsv[:,:,z], None)[1]
        vector_orb = np.zeros(c_n)
        try:
            for f in features_orb:
                vector_orb[((visual_words_orb_hsv[z] - f)**2).sum(axis=1).argmin()] += 1
        except:
                pass
        Vector_orb_hsv[z][i,:] = vector_orb
    #############################################################
#Colum names bag of features by Orb
n_orb_gry, n_orb_l, n_orb_a, n_orb_bl, n_orb_h, n_orb_s, n_orb_v = ['orb_gry']*c_n, ['orb_l']*c_n, ['orb_a']*c_n, ['orb_bl']*c_n,['orb_h']*c_n, ['orb_s']*c_n, ['orb_v']*c_n
n_orb_b, n_orb_g, n_orb_r = ['orb_b']*c_n, ['orb_g']*c_n, ['orb_r']*c_n

number = list(range(1,c_n+1))
number_list = list(map(str, number))
orb_gry_list = [a + b for a, b in zip(n_orb_gry, number_list)]
orb_b_list = [a + b for a, b in zip(n_orb_b, number_list)]
orb_g_list = [a + b for a, b in zip(n_orb_g, number_list)]
orb_r_list = [a + b for a, b in zip(n_orb_r, number_list)]
orb_l_list = [a + b for a, b in zip(n_orb_l, number_list)]
orb_a_list = [a + b for a, b in zip(n_orb_a, number_list)]
orb_bl_list = [a + b for a, b in zip(n_orb_bl, number_list)]
orb_h_list = [a + b for a, b in zip(n_orb_h, number_list)]
orb_s_list = [a + b for a, b in zip(n_orb_s, number_list)]
orb_v_list = [a + b for a, b in zip(n_orb_v, number_list)]
feature_orb_gry = pd.DataFrame(Vector_orb_gry, columns = orb_gry_list)
feature_orb_b = pd.DataFrame(Vector_orb_bgr[0], columns = orb_b_list)
feature_orb_g = pd.DataFrame(Vector_orb_bgr[1], columns = orb_g_list)
feature_orb_r = pd.DataFrame(Vector_orb_bgr[2], columns = orb_r_list)
feature_orb_L = pd.DataFrame(Vector_orb_Lab[0], columns = orb_l_list)
feature_orb_a = pd.DataFrame(Vector_orb_Lab[1], columns = orb_a_list)
feature_orb_bl = pd.DataFrame(Vector_orb_Lab[2], columns = orb_bl_list)
feature_orb_h = pd.DataFrame(Vector_orb_hsv[0], columns = orb_h_list)
feature_orb_s = pd.DataFrame(Vector_orb_hsv[1], columns = orb_s_list)
feature_orb_v = pd.DataFrame(Vector_orb_hsv[2], columns = orb_v_list)

col=[]
col.extend(orb_gry_list + orb_b_list + orb_g_list + orb_r_list + orb_l_list + orb_a_list + orb_bl_list + orb_h_list + orb_s_list + orb_v_list)
df_train = pd.concat([pd.DataFrame(y_train.values, columns=['label']),pd.DataFrame(X_train.values, columns=df.columns[1:]),
                      feature_orb_gry, feature_orb_b, feature_orb_g, feature_orb_r, feature_orb_L, feature_orb_a, feature_orb_bl, feature_orb_h, feature_orb_s, feature_orb_v],axis=1)
#Save training data set
df_train.to_csv('data/train_feature_dummy.csv')


C:\Users\prajj\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:888: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 1024 or by setting the environment variable OMP_NUM_THREADS=1
  f"MiniBatchKMeans is known to have a memory leak on "
C:\Users\prajj\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:888: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 1024 or by setting the environment variable OMP_NUM_THREADS=1
  f"MiniBatchKMeans is known to have a memory leak on "
C:\Users\prajj\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:888: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 1024 or by 

In [12]:
pi = df_train.drop(['label'],axis=1)
pi

,gry,b,g,r,h,s,v,l,a,bl,...,orb_v11,orb_v12,orb_v13,orb_v14,orb_v15,orb_v16,orb_v17,orb_v18,orb_v19,orb_v20
0,77.660581,30.857381,94.180801,63.059652,44.864168,172.803123,94.278079,92.614820,104.996656,159.581385,...,29.0,33.0,51.0,64.0,3.0,8.0,17.0,4.0,2.0,3.0
1,62.720407,42.184169,70.275404,55.717622,45.961527,106.448614,70.275590,70.900280,116.240467,143.057940,...,30.0,6.0,10.0,10.0,16.0,29.0,10.0,22.0,8.0,20.0
2,80.326384,40.979810,93.471476,69.534754,43.868057,145.124823,93.471616,93.430560,109.205087,154.812009,...,10.0,6.0,38.0,13.0,8.0,38.0,9.0,27.0,4.0,26.0
3,83.149405,39.537043,96.813202,72.971714,42.667931,152.732691,96.833589,96.787394,108.696575,157.173632,...,37.0,32.0,32.0,26.0,10.0,8.0,13.0,7.0,16.0,6.0
4,94.276226,42.454256,112.134995,78.996643,44.473627,159.367023,112.164350,110.794479,103.673175,161.998836,...,14.0,16.0,45.0,82.0,3.0,12.0,14.0,8.0,3.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1040,62.697178,40.068548,70.705808,55.628056,45.243788,112.724331,70.738715,71.173945,115.620566,144.432143,...,15.0,37.0,16.0,24.0,12.0,34.0,9.0,13.0,7.0,18.0
1041,62.511272,27.716010,73.922049,53.440430,43.647389,160.930873,73.936583,73.034478,111.195741,152.398550,...,17.0,9.0,17.0,26.0,20.0,33.0,16.0,24.0,5.0,30.0
1042,86.067174,32.462627,103.112960,73.056319,42.574733,179.744107,103.559482,101.901240,104.875254,162.698629,...,43.0,81.0,31.0,24.0,26.0,19.0,17.0,12.0,8.0,27.0
1043,88.216684,36.115662,105.765934,73.640716,44.088891,168.564780,105.802422,104.408431,104.051653,162.173049,...,48.0,23.0,26.0,35.0,4.0,18.0,4.0,8.0,4.0,7.0


In [4]:
import joblib
loaded_model = joblib.load('models_grid.pkl')

In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(pi)
scale_train =scaler.transform(pi)

In [17]:
tim=loaded_model.predict(scale_train)

In [29]:
count=0
for i in range(1045):
    if tim[i]!=label[i][0]:
       count+=1
    
count    

481

In [32]:
pimp=feature_color
scaler = StandardScaler()
scaler.fit(pimp)
scale_train2 =scaler.transform(pimp)

In [34]:
X_train

,gry,b,g,r,h,s,v,l,a,bl,...,gry_min,b_min,g_min,r_min,h_min,s_min,v_min,l_min,a_min,bl_min
0,77.660581,30.857381,94.180801,63.059652,44.864168,172.803123,94.278079,92.614820,104.996656,159.581385,...,29.0,1.0,40.0,18.0,19.0,51.0,40.0,35.0,95.0,141.0
1,62.720407,42.184169,70.275404,55.717622,45.961527,106.448614,70.275590,70.900280,116.240467,143.057940,...,18.0,1.0,25.0,10.0,28.0,10.0,25.0,18.0,101.0,123.0
2,80.326384,40.979810,93.471476,69.534754,43.868057,145.124823,93.471616,93.430560,109.205087,154.812009,...,31.0,2.0,44.0,17.0,24.0,26.0,44.0,38.0,98.0,131.0
3,83.149405,39.537043,96.813202,72.971714,42.667931,152.732691,96.833589,96.787394,108.696575,157.173632,...,21.0,1.0,25.0,18.0,14.0,26.0,25.0,19.0,98.0,128.0
4,94.276226,42.454256,112.134995,78.996643,44.473627,159.367023,112.164350,110.794479,103.673175,161.998836,...,41.0,1.0,54.0,28.0,21.0,23.0,54.0,50.0,96.0,138.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1040,62.697178,40.068548,70.705808,55.628056,45.243788,112.724331,70.738715,71.173945,115.620566,144.432143,...,10.0,1.0,14.0,5.0,18.0,19.0,14.0,8.0,105.0,120.0
1041,62.511272,27.716010,73.922049,53.440430,43.647389,160.930873,73.936583,73.034478,111.195741,152.398550,...,10.0,1.0,12.0,4.0,12.0,8.0,12.0,8.0,100.0,124.0
1042,86.067174,32.462627,103.112960,73.056319,42.574733,179.744107,103.559482,101.901240,104.875254,162.698629,...,26.0,1.0,35.0,18.0,17.0,10.0,35.0,29.0,95.0,132.0
1043,88.216684,36.115662,105.765934,73.640716,44.088891,168.564780,105.802422,104.408431,104.051653,162.173049,...,24.0,1.0,35.0,13.0,20.0,48.0,35.0,28.0,96.0,139.0
